In [1]:
#%pip install pyaudio -q
%pip install --upgrade pip -q
%pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 -q
%pip install youtube-transcript-api bs4 pypdf -q
%pip install SpeechRecognition -q 
%pip install opencv-python -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import WebBaseLoader, YoutubeLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pytube import YouTube
from web3 import Web3

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
############################# Document instances
## Create a crawl to list all the webpages, and then load them here
web1 = WebBaseLoader("https://www.fidesinnova.io")
web2 = WebBaseLoader("https://fidesinnova.io/devices/")
web3 = WebBaseLoader("https://fidesinnova.io/Contacts/")


## Adding all repositories
git1 = WebBaseLoader("https://github.com/TheArchitect2000/iot-server")
git2 = WebBaseLoader("https://github.com/TheArchitect2000/zkiot-riscv-qemu-c")
git3 = WebBaseLoader("https://github.com/TheArchitect2000/Fides-Innova-WiKi")


## Addming all PDfs
pdf1 = PyPDFLoader("zkIoT.pdf")

## Adding all YouTube videos
video1 = YoutubeLoader("kgYxyxeDNl4")
video2 = YoutubeLoader("VZMBE2NLSC4")
video3 = YoutubeLoader("YpfFHI3Ivmo")


## Fides Blockchain
rpc_url = "https://rpc1.fidesinnova.io"
w3 = Web3(Web3.HTTPProvider(rpc_url))

if w3.is_connected():
    print("✅ Connected to FidesInnova Blockchain!")
    print("Latest block:", w3.eth.block_number)
else:
    print("❌ Connection failed.")

contract_address = Web3.to_checksum_address("0x4b08ea934e6bfb7c72a376c842c911e1dd2aa74f")
with open("NodeServiceDeviceManagement.abi") as f1:
   abi =  f1.read() # Smart contract ABI

contract = w3.eth.contract(address=contract_address, abi=abi)

#sensor_data = contract.functions.getSensorData(sensor_id).call()

############################# Loading
loaders = [web1, web2, web3, git1, git2, git3, pdf1, video1, video2, video3]

docs = []
for loader in loaders:
    loaded_docs = loader.load()
    for doc in loaded_docs:
        if isinstance(loader, YoutubeLoader):
            doc.metadata['type'] = 'YouTube'
        elif isinstance(loader, PyPDFLoader):
            doc.metadata['type'] = 'PDF'
        elif isinstance(loader, WebBaseLoader):
            doc.metadata['type'] = 'Web'
        else:
            doc.metadata['type'] = 'Other'
        print(doc.metadata)
        docs.append(doc)

✅ Connected to FidesInnova Blockchain!
Latest block: 3169687
{'source': 'https://www.fidesinnova.io', 'title': 'FidesInnova-Verified Computations for Reliable Results - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://fidesinnova.io/devices/', 'title': 'Devices - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://fidesinnova.io/Contacts/', 'title': 'Contacts - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://github.com/TheArchitect2000/iot-server', 'title': 'GitHub - TheArchitect2000/iot-server: An IoT Management System for registering IoT devices, communicating via the MQTT protocol, managing users, and more.', 'description': 'An IoT Management System for registering IoT devices, communicating via the MQTT protocol, managing users, and more. - TheArchitect2000/iot-server', 'language': 'en', 'type': 'Web'}
{'source': 'https://github.com/TheArchitect2000/zkiot-riscv-qemu-c', 'title': 'GitHub - TheArchitect2000/zkiot-riscv-qemu

In [4]:
############################# split the documents
splitter1 = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
totaldoc2 = splitter1.split_documents(docs)

############################# Creating Vector database
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

api_key1 = "***REMOVED***"
embedding1 = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key1)

# on RAM memory
#mydb1 = Chroma.from_documents(totaldoc2, embedding=embedding1) 
# on storage
mydb1 = Chroma(
    collection_name="example_collection",
    embedding_function=embedding1,
    persist_directory="chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

mydb1.add_documents(totaldoc2)

/var/folders/dx/hkx51pqs1vxc2zsq74nxsjp80000gn/T/ipykernel_8384/1670620466.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  mydb1 = Chroma(


['13a14c8f-46b9-4f70-a94c-b5a414e5c4d7',
 '5c1c521b-7b70-407c-bbc5-4022d24f885a',
 '279aa88d-7f4d-4fca-81ea-fe7a8ae398bc',
 '32e3ef4a-7db8-4832-8c31-d9e52453f58e',
 '6d2622a7-21c5-4d5b-a40c-1d59f7c066e5',
 '36771664-adeb-4cd9-848b-188fe75c753a',
 '4744cc2f-3102-4624-9911-a2bcaef120b3',
 'bb38584a-5f59-4f85-8a38-a7af9a714403',
 'ce224f0d-4a78-4b91-966f-044c040f8b3f',
 '0d1a3b93-f55f-4bdc-8540-a5c508557454',
 '4127ec32-1d3f-40d7-88c3-bf91855f6069',
 'ca2151a1-164d-4a02-a2b7-52e1f65af5e8',
 '5e6d73c3-bb6f-4db4-8793-c50b38eda9cf',
 'f6108e97-f63d-4c9d-ba31-b9f125ca9fac',
 '5267a69b-b673-4c83-86d8-b787401a75f8',
 'c4b936d3-0008-475f-8065-be467f4de2c8',
 'dfabee3d-73dc-4ed5-89ae-7ddfdebb4102',
 '95b65c32-ee8a-4af8-81d9-d593d30f299a',
 '3645fb79-ed52-451a-9ffe-56336a92412d',
 '9af85a83-1958-4c45-b688-ee8cf43610c4',
 '12481997-a81b-44f6-9f74-c9c578b823ea',
 '8cb7712d-ca5f-4fda-90a5-219ff637c627',
 '3a6a33ff-2bf0-44c6-8d28-b13929cf605b',
 '3b192efb-24ff-4dbe-97ab-dd66181b09a3',
 '200dd7d2-528e-

In [ ]:
# with open("a.txt","w") as f1:
#     f1.write(web1.load()[0].page_content)